In [ ]:
%load_ext kamu

In [ ]:
%%local
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import holoviews as hv
from utils import plotting
from mapboxgl.viz import *
from mapboxgl.utils import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

mapbox_style = 'mapbox://styles/mapbox/streets-v11'

In [ ]:
%import_dataset com.arcgis.hub.usa-counties --alias usa_counties

In [ ]:
# By default code runs PySpark on the server side
# Use %%local tag to execute code in the local notebook kernel
usa_counties.printSchema()

In [ ]:
%%sql
select * from usa_counties limit 3

In [ ]:
%%sql -o df_countries -q
select * from usa_counties

In [ ]:
%%local
viz = ChoroplethViz(
    plotting.df_to_geojson(df_countries),
    style=mapbox_style,
    center=(-101, 41),
    zoom=3,
    access_token=token,
    color_property='gis_area',
    color_stops=create_color_stops([0], colors=['#8888ff']),
    color_default='white',
    line_width=0,
    opacity=0.2,
)

viz.show()

In [ ]:
%import_dataset com.weatherxm --alias weatherxm

In [ ]:
%%sql
select * from weatherxm limit 5

In [ ]:
%%sql
select
    event_time,
    precipitation_accumulated
from weatherxm
where device_id = "78243940-96aa-11ed-9972-4f669f2d96bd" and precipitation_accumulated is not null

In [ ]:
%%sql -o df -q
with all_measurements as (
    select
        event_time,
        device_id,
        lat,
        lon,
        precipitation_accumulated as p
    from weatherxm
    where precipitation_accumulated is not null
),
by_device as (
    select
        device_id,
        avg(lat) as lat,
        avg(lon) as lon,
        max(p) - min(p) as p_delta
    from all_measurements
    group by 1
    having p_delta < 1000 -- dirty hack to remove outliers
)
select * from by_device

In [ ]:
%%local
df.hvplot.hist("p_delta", bins=100)

In [ ]:
%%local
viz = CircleViz(
    df_to_geojson(
        df, 
        properties=['p_delta'],
        lat='lat',
        lon='lon',
        precision=3
    ),
    style=mapbox_style,
    center=(7,50),
    zoom=4,
    radius=4,
    color_property='p_delta',
    color_stops=create_color_stops([0,5,10,15,20,25,30,100], colors='Blues'),
    stroke_width=0.1,
    opacity=0.8
)

viz.show()

In [ ]:
%%sql -o df_precip_by_county -q
with all_measurements as (
    select
        event_time,
        device_id,
        lat,
        lon,
        precipitation_accumulated as p
    from weatherxm
    where precipitation_accumulated is not null
),
by_device as (
    select
        device_id,
        avg(lat) as lat,
        avg(lon) as lon,
        max(p) - min(p) as p_delta
    from all_measurements
    group by 1
    having p_delta < 1000 -- dirty hack to remove outliers
),
by_county as (
    select
        county.fips,
        first(county.name) as name,
        count(*) as num_devices,
        avg(p_delta) as avg_precip
    from
        (
            select
                fips,
                name,
                st_geomfromgeojson(geometry) as geometry
            from usa_counties
        ) as county,
        by_device as dev
    where st_contains(county.geometry, st_point(dev.lon, dev.lat))
    group by 1
),
by_county_geoms as (
    select
        c.fips,
        c.name,
        c.geometry,
        bc.num_devices,
        bc.avg_precip
    from usa_counties as c
    left join by_county as bc
    on c.fips = bc.fips
)
select * from by_county_geoms

In [ ]:
%%local
df_precip_by_county = df_precip_by_county.replace({np.nan: 0})

viz = ChoroplethViz(
    plotting.df_to_geojson(df_precip_by_county),
    style=mapbox_style,
    center=(-81, 34),
    zoom=4,
    access_token=token,
    color_property='avg_precip',
    color_stops=create_color_stops([0,1,3,5,10,20], colors='Oranges'),
    color_default='#ffffff',
    line_width=0.4,
    line_color='#aaaaff',
    opacity=0.3,
)

viz.show()